In [1]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

import torch
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModelForMaskedLM
from multiprocessing import Pool, cpu_count
from transformers import BertTokenizer, BertForSequenceClassification

os.environ['CUDA_VISIBLE_DEVICES'] = "3"

ENTITY_INPUT_PATH = './data/3_openKE/synthesize/entity2id.txt'
ENTITY_INPUT_LABEL_PATH = './data/3_openKE/synthesize/entity2id_objType.txt'

ENTITY_OUTPUT_PATH = './data/4_embedding/synthesize/secureBERT'
MODEL_SAVE_PATH = './data/4_embedding/synthesize/model/secureBERT/entity/'

In [2]:
X = list()
Y = list()
with open(ENTITY_INPUT_LABEL_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        cat, eid = row.split('\t')
        Y.append(cat)
with open(ENTITY_INPUT_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        value, eid = row.split('\t')
        X.append(value)

print(f'Sentence Cnt: {len(X)}' , X[:5])
print(f'Label Cnt: {len(Y)}', Y[:5])

Sentence Cnt: 500281 ['Device\\HarddiskVolume2\\Qt\\Qt5.10.1\\5.10.1\\Src\\qtbase\\src\\plugins\\platforms\\cocoa\\qcocoamimetypes.mm', '\\Device\\HarddiskVolume2\\Qt\\Qt5.10.1\\5.10.1\\Src\\qtbase\\tests\\auto\\corelib\\xml\\qxmlstream\\XML-Test-Suite\\xmlconf\\ibm\\valid\\P85\\out\\CVS\\Root', '53&128.55.12.10&128.55.12.67&17&55258', '\\Device\\HarddiskVolume2\\Qt\\Qt5.10.1\\Examples\\Qt-5.10.1\\location\\places\\forms\\SearchBoundingBox.qml', '\\REGISTRY\\USER\\S-1-5-21-231540947-922634896-4161786520-1004_Classes\\Local_Settings\\Software\\Microsoft\\Windows\\CurrentVersion\\AppContainer\\Storage\\microsoft.windows.shellexperiencehost_cw5n1h2txyewy']
Label Cnt: 500281 ['file', 'file', 'network', 'file', 'registry']


In [3]:
for sentence, label in zip(X, Y):
    if label == 'network':
        print(label, sentence)

network 53&128.55.12.10&128.55.12.67&17&55258
network 54715&128.55.12.67&128.55.12.10&17&53
network 5355&ff02::1:3&fe80::2544:d52e:aa5e:4967&17&62356
network 59217&0.0.0.0&&6&0
network 5355&224.0.0.252&128.55.12.141&17&62885
network 59764&0.0.0.0&&6&0
network 57683&128.55.12.67&64.233.169.103&6&443
network 65014&128.55.12.67&64.233.169.103&6&443
network 60315&0.0.0.0&&6&0
network 58883&10.0.6.91&224.0.0.252&17&5355
network 56703&128.55.12.67&128.55.12.10&17&53
network 62946&0.0.0.0&&6&0
network 55318&::&&17&0
network 50450&::&&17&0
network 62600&::&&17&0
network 53998&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&61387
network 51084&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&54591
network 53&128.55.12.10&128.55.12.67&17&49281
network 5900&128.55.12.67&128.55.12.122&6&34288
network 50259&128.55.12.67&209.85.197.147&6&443
network 53&128.55.12.10&128.55.12.67&17&63471
network 60168&128.55.12.67&64.233.169.103&6&443
network 53&128.55.12.10&128.55.12.67&1

network 50581&0.0.0.0&&6&0
network 53874&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&62782
network 50891&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&51144
network 64263&128.55.12.67&198.151.60.100&6&443
network 64230&::&&17&0
network 53&128.55.12.10&128.55.12.67&17&60929
network 59551&::&&17&0
network 53&128.55.12.10&128.55.12.67&17&65071
network 60612&0.0.0.0&&6&0
network 55812&128.55.12.67&216.163.248.17&6&80
network 55433&::&&17&0
network 60878&128.55.12.67&64.233.169.103&6&443
network 59175&128.55.12.67&128.55.12.10&17&53
network 50241&128.55.12.67&128.55.12.10&17&53
network 58670&128.55.12.67&128.55.12.10&17&53
network 54136&::&&17&0
network 53591&::&&17&0
network 50932&::&&17&0
network 64647&128.55.12.162&224.0.0.252&17&5355
network 58619&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&51888
network 58418&128.55.12.67&64.233.169.103&6&443
network 59324&0.0.0.0&&6&0
network 5355&ff02::1:3&fe80::2544:d5

network 52264&0.0.0.0&&6&0
network 60721&128.55.12.67&64.233.169.103&6&443
network 52273&128.55.12.67&8.12.96.83&6&80
network 57149&128.55.12.67&128.55.12.10&17&53
network 65407&::&&17&0
network 5900&128.55.12.67&128.55.12.122&6&49124
network 65484&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&61122
network 57287&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&64566
network 65378&0.0.0.0&&6&0
network 57831&128.55.12.67&208.17.90.10&6&80
network 51786&128.55.12.67&137.78.99.23&6&443
network 62863&::&&17&0
network 54182&128.55.12.67&204.2.179.67&6&80
network 56964&0.0.0.0&&6&0
network 56816&128.55.12.67&128.55.12.10&17&53
network 55604&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&53857
network 53&128.55.12.10&128.55.12.67&17&56278
network 61106&0.0.0.0&&6&0
network 61154&128.55.12.67&69.147.114.210&6&443
network 49367&::&&17&0
network 49386&0.0.0.0&&6&0
network 52658&::&&17&0
network 58483&128.55.12.67&216.163.248.17&6&80
network 52801&0.0.0.0&&6&0
network 61684&::&&1

network 52723&0.0.0.0&&6&0
network 53320&::&&17&0
network 60161&128.55.12.67&128.55.12.10&17&53
network 61960&128.55.12.67&64.233.169.103&6&443
network 58471&::&&17&0
network 65140&128.55.12.67&216.163.248.17&6&80
network 51715&128.55.12.67&67.28.122.168&6&80
network 58960&0.0.0.0&&6&0
network 55584&::&&17&0
network 57551&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&65193
network 54837&0.0.0.0&&6&0
network 50355&128.55.12.67&128.55.12.10&17&53
network 54124&128.55.12.67&162.97.114.199&6&80
network 65289&128.55.12.67&128.55.12.10&17&53
network 55006&::&&17&0
network 63274&128.55.12.67&128.55.12.10&17&53
network 57740&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&52843
network 63337&::&&17&0
network 60959&::&&17&0
network 51460&0.0.0.0&&6&0
network 53416&::&&17&0
network 61169&128.55.12.67&207.138.235.231&6&80
network 60217&::&&17&0
network 65187&::&&17&0
network 51311&128.55.12.67&128.55.12.10&17&53
network 64684&128.55.12.67&203.146.

network 59416&128.55.12.67&128.55.12.10&17&53
network 445&128.55.12.67&128.55.12.110&6&57543
network 65396&128.55.12.67&128.55.12.10&17&53
network 56353&128.55.12.67&128.55.12.10&17&53
network 59870&128.55.12.67&69.147.114.210&6&443
network 61734&0.0.0.0&&6&0
network 58235&0.0.0.0&&6&0
network 52311&0.0.0.0&&6&0
network 55227&0.0.0.0&&6&0
network 60110&::&&17&0
network 50774&128.55.12.67&204.2.179.67&6&80
network 61120&128.55.12.67&64.233.169.103&6&443
network 57730&128.55.12.67&161.207.1.180&6&80
network 51174&0.0.0.0&&6&0
network 58807&128.55.12.67&64.233.169.103&6&443
network 53426&128.55.12.67&128.55.12.10&17&53
network 52600&128.55.12.67&158.57.45.15&6&80
network 62234&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&55585
network 54467&0.0.0.0&&6&0
network 61425&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&56085
network 63223&::&&17&0
network 52909&128.55.12.67&216.148.223.72&6&80
network 55112&128.55.12.67&207.217.125.95&6&80
network 64542&128.55.1

...

network 5900&128.55.12.67&128.55.12.122&6&48140
network 58525&128.55.12.67&64.233.169.103&6&443
network 59163&0.0.0.0&&6&0
network 54194&128.55.12.67&204.2.179.67&6&80
network 50220&128.55.12.67&128.55.12.10&17&53
network 52080&128.55.12.67&128.55.12.10&17&53
network 56586&::&&17&0
network 61248&0.0.0.0&&6&0
network 5355&ff02::1:3&fe80::2544:d52e:aa5e:4967&17&55388
network 54474&::&&17&0
network 53&128.55.12.10&128.55.12.67&17&50577
network 58401&128.55.12.67&69.147.114.210&6&443
network 50765&128.55.12.67&208.75.170.1&6&80
network 55004&128.55.12.67&128.55.12.10&17&53
network 51735&128.55.12.67&206.16.45.70&6&80
network 53&128.55.12.10&128.55.12.67&17&56914
network 53&128.55.12.10&128.55.12.67&17&62324
network 62597&128.55.12.67&64.154.82.134&6&80
network 53&128.55.12.10&128.55.12.67&17&61838
network 55390&0.0.0.0&&6&0
network 63011&0.0.0.0&&6&0
network 59579&::&&17&0
network 53749&128.55.12.67&64.233.169.103&6&443
network 49662&::&&17&0
network 63408&10.0.6.29&224.0.0.252&17&5355
net

network 53984&128.55.12.67&128.55.12.10&17&53
network 64847&128.55.12.67&128.55.12.10&17&53
network 63764&128.55.12.67&128.55.12.10&17&53
network 59542&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&58957
network 54300&::&&17&0
network 52283&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&56275
network 49421&::&&17&0
network 53&128.55.12.10&128.55.12.67&17&58745
network 61206&::&&17&0
network 58587&128.55.12.67&128.55.12.10&17&53
network 64581&::&&17&0
network 50461&128.55.12.67&207.217.125.95&6&80
network 54757&::&&17&0
network 49312&0.0.0.0&&6&0
network 5900&128.55.12.67&128.55.12.122&6&42536
network 50625&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&60267
network 52752&128.55.12.67&128.55.12.10&17&53
network 5355&ff02::1:3&fe80::2544:d52e:aa5e:4967&17&60791
network 56921&0.0.0.0&&6&0
network 59422&128.55.12.67&64.154.82.134&6&80
network 64462&128.55.12.67&64.233.169.103&6&443
network 53&128.55.12.10&128.55.12.67&17&56262
network 53&128.55.12.10&

network 56247&0.0.0.0&&6&0
network 60788&0.0.0.0&&6&0
network 53346&0.0.0.0&&6&0
network 59941&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&55371
network 49195&0.0.0.0&&6&0
network 64706&128.55.12.67&216.148.223.72&6&80
network 64959&128.55.12.67&208.17.90.10&6&80
network 50602&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&59584
network 60660&128.55.12.67&64.154.82.134&6&80
network 5900&128.55.12.67&128.55.12.122&6&48068
network 52042&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&61640
network 52140&::&&17&0
network 60658&0.0.0.0&&6&0
network 53942&::&&17&0
network 59650&128.55.12.67&128.55.12.10&17&53
network 51750&::&&17&0
network 56012&::&&17&0
network 65457&128.55.12.67&128.55.12.10&17&53
network 58857&128.55.12.67&128.55.12.10&17&53
network 54021&128.55.12.67&63.240.4.179&6&443
network 65112&128.55.12.67&216.163.248.17&6&80
network 63355&::&&17&0
network 59500&128.55.12.67&64.233.169.103&6&443
network 62935&128.55.12.67&128.55.12.10&17&53
n

network 64316&128.55.12.67&64.233.169.103&6&443
network 64611&128.55.12.67&161.207.1.180&6&80
network 52410&0.0.0.0&&6&0
network 56696&128.55.12.67&128.55.12.10&17&53
network 51177&0.0.0.0&&6&0
network 49217&128.55.12.67&128.55.12.10&17&53
network 54234&::&&17&0
network 60104&::&&17&0
network 62972&0.0.0.0&&6&0
network 61198&128.55.12.67&128.55.12.10&17&53
network 57176&128.55.12.67&128.55.12.10&17&53
network 51612&128.55.12.67&64.233.169.103&6&443
network 49482&::&&17&0
network 53432&128.55.12.67&74.52.178.194&6&80
network 62698&128.55.12.67&128.55.12.10&17&53
network 53&128.55.12.10&128.55.12.67&17&62651
network 55737&::&&17&0
network 50314&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&59736
network 62486&::&&17&0
network 65298&0.0.0.0&&6&0
network 60870&128.55.12.67&69.147.114.210&6&443
network 59250&0.0.0.0&&6&0
network 58066&0.0.0.0&&6&0
network 50052&128.55.12.67&128.55.12.10&17&53
network 53383&128.55.12.67&64.233.169.103&6&443
network 59839&128.55.12.67&128.55.12.10&17&5

network 58746&128.55.12.67&128.55.12.10&17&53
network 51051&0.0.0.0&&6&0
network 51400&128.55.12.67&208.17.90.10&6&80
network 55743&0.0.0.0&&6&0
network 61199&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&53674
network 50909&0.0.0.0&&6&0
network 54307&128.55.12.67&128.55.12.10&17&53
network 53373&::&&17&0
network 58481&::&&17&0
network 52535&128.55.12.67&64.233.169.103&6&443
network 58369&0.0.0.0&&6&0
network 53&128.55.12.10&128.55.12.67&17&60934
network 61010&128.55.12.67&207.138.235.231&6&80
network 53607&0.0.0.0&&6&0
network 57156&128.55.12.67&203.146.102.130&6&80
network 53&128.55.12.10&128.55.12.67&17&52457
network 62633&::&&17&0
network 54698&128.55.12.67&64.233.169.103&6&443
network 53&128.55.12.10&128.55.12.67&17&52530
network 62024&128.55.12.67&64.233.169.103&6&443
network 55179&0.0.0.0&&6&0
network 60116&128.55.12.67&128.55.12.10&17&53
network 60071&128.55.12.67&64.233.169.103&6&443
network 53&128.55.12.10&128.55.12.67&17&63708
network 53&128.55.12.10&128.55.12.67&17&5

In [4]:
# Convert category labels to tensor
label_dict = {'file':0, 'registry':1, 'network':2, 'process':3}
labels = torch.tensor([label_dict[y] for y in Y])

In [5]:
# SecureBERT
tokenizer = RobertaTokenizer.from_pretrained("ehsanaghaei/SecureBERT")
sentence = "C:\\Users\\ezk\\Anaconda3\\pkgs\\libffi-3.4.2-hd77b12b_4\\info\\test\\testsuite\\libffi.complex\\return_complex.inc"
input_ids = tokenizer(sentence)["input_ids"]
token_words = tokenizer.convert_ids_to_tokens(input_ids)
print(token_words)
sentence = "HKLM\\System\\CurrentControlSet\\Enum\\USB\\VID_8564&PID_1000\\76UXHGBEGVWDAVZZ\\Properties\\{3b2ce006-5e61-4fde-bab8-9b8aac9b26df}\\0008"
input_ids = tokenizer(sentence)["input_ids"]
token_words = tokenizer.convert_ids_to_tokens(input_ids)
print(token_words)
sentence = '82666&powershell.exe_-ExecutionPolicy_Bypass_-C_"powershell.exe_-c_\\"Get-WmiObject_-class_win32_operatingsystem_|_select_-property_*_|_export-csv_msdebug.log\\";"'
input_ids = tokenizer(sentence)["input_ids"]
token_words = tokenizer.convert_ids_to_tokens(input_ids)
print(token_words)
sentence = '54345&get-wmiobject_win32_computersystem_|_fl_model'
input_ids = tokenizer(sentence)["input_ids"]
token_words = tokenizer.convert_ids_to_tokens(input_ids)
print(token_words)
sentence = '65020&wevtutil__sl_"Microsoft-Windows-IKE/Operational"_/e:false'
input_ids = tokenizer(sentence)["input_ids"]
token_words = tokenizer.convert_ids_to_tokens(input_ids)
print(token_words)

['<s>', 'C', ':\\', 'Users', '\\', 'ez', 'k', '\\', 'An', 'ac', 'onda', '3', '\\', 'pkg', 's', '\\', 'lib', 'ff', 'i', '-', '3', '.', '4', '.', '2', '-', 'hd', '77', 'b', '12', 'b', '_', '4', '\\', 'info', '\\', 'test', '\\', 'test', 'suite', '\\', 'lib', 'ff', 'i', '.', 'complex', '\\', 'return', '_', 'complex', '.', 'inc', '</s>']
['<s>', 'H', 'KLM', '\\', 'System', '\\', 'CurrentControlSet', '\\', 'Enum', '\\', 'USB', '\\', 'VID', '_', '85', '64', '&', 'PID', '_', '1000', '\\', '76', 'UX', 'H', 'GB', 'EG', 'V', 'WD', 'AV', 'ZZ', '\\', 'Proper', 'ties', '\\{', '3', 'b', '2', 'ce', '006', '-', '5', 'e', '61', '-', '4', 'f', 'de', '-', 'bab', '8', '-', '9', 'b', '8', 'aac', '9', 'b', '26', 'df', '}\\', '000', '8', '</s>']
['<s>', '8', '26', '66', '&', 'powers', 'hell', '.', 'exe', '_', '-', 'Execution', 'Policy', '_', 'B', 'ypass', '_', '-', 'C', '_', '"', 'powers', 'hell', '.', 'exe', '_', '-', 'c', '_', '\\"', 'Get', '-', 'W', 'mi', 'Object', '_', '-', 'class', '_', 'win', '32', '_',

In [6]:
# SecureBERT
# tokenizer = RobertaTokenizer.from_pretrained("ehsanaghaei/SecureBERT")
encoded_inputs = tokenizer.batch_encode_plus(
    X,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

In [7]:
print(input_ids.shape)
print(attention_mask.shape)

torch.Size([500281, 128])
torch.Size([500281, 128])


In [8]:
# Move data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

In [9]:
batch_size = 16

In [10]:
# splitter_tmp = int(0.9 * len(input_ids))
splitter1 = int(0.8 * len(input_ids))
train_inputs, val_inputs= input_ids, input_ids[splitter1:]
train_masks, val_masks= attention_mask, attention_mask[splitter1:]
train_labels, val_labels = labels, labels[splitter1:]
print(train_inputs.shape, Counter(Y))
print(val_inputs.shape, Counter(Y[splitter1:]))

# Create data loaders
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

torch.Size([500281, 128]) Counter({'file': 398474, 'network': 74941, 'registry': 22790, 'process': 4076})
torch.Size([100057, 128]) Counter({'file': 79799, 'network': 14827, 'registry': 4614, 'process': 817})


In [11]:
model = BertForSequenceClassification.from_pretrained("ehsanaghaei/SecureBERT", num_labels=4, output_hidden_states=True)
# Move the model to GPU
model.to('cuda')

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ehsanaghaei/SecureBERT were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.1.attention.self.value.bias', 'roberta.encoder.layer.9.attention.output.dense.bias', 'roberta.encoder.layer.2.attention.self.key.weight', 'roberta.encoder.layer.11.attention.output.LayerNorm.weight', 'roberta.encoder.layer.4.output.dense.bias', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.10.output.dense.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.encoder.layer.6.intermediate.dense.bias', 'roberta.encoder.layer.3.attention.self.value.bias', 'roberta.encoder.layer.7.intermediate.dense.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.10.attention.output.dense.bias', 'roberta.encoder.layer.11.output.dense.weight',

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['encoder.layer.9.attention.self.key.bias', 'encoder.layer.9.attention.self.value.bias', 'classifier.bias', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.1.attention.output.LayerNorm.weight', 'encoder.layer.5.attention.self.query.weight', 'encoder.layer.11.intermediate.dense.weight', 'encoder.layer.9.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.9.attention.self.query.bias', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.10.attention.output.dense.bias', 'encoder.layer.11.output.dense.weight', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.1.attention.output.dense.weight', 'encoder.layer.6.attention.self.query.weight', 'encoder.layer.8.attention.self.value.weight', 'encoder.layer.2.output.dense.weight', 'encoder.layer.10.attention.self.value.weight', 'encoder.laye

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [12]:
len(train_dataloader)

31268

In [13]:
# Fine-tune the BERT model on my task:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
epochs = 1
batch_chunk_size = 10000  # Number of batches to process before saving to disk

# Training loop
min_val_loss = 999999
for epoch in range(epochs):
    model.train()
    embeddings = []
    print(f'Epoch {epoch+1} start!!!!!!!')
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        
        # Extract the embeddings from the model's hidden states
        hidden_states = model.bert(input_ids, attention_mask=attention_mask)[2]
        last_hidden_layer_embeddings = hidden_states[-1].detach().cpu().numpy()
        embeddings.append(last_hidden_layer_embeddings)
        
        if ((batch_idx+1) % batch_chunk_size == 0) or batch_idx == len(train_dataloader)-1:
            # Reshape embeddings to (n_sentences, embedding_dim)
            embeddings = np.concatenate(embeddings)
            embeddings = embeddings[:, 0, :]  # Take the embedding of the [CLS] token
            embeddings = np.array(embeddings)
            np.save(f'{ENTITY_OUTPUT_PATH}/embeddings_chunk_{batch_idx // batch_chunk_size}.npy', embeddings)
            print(f'Saved {ENTITY_OUTPUT_PATH}/embeddings_chunk_{batch_idx // batch_chunk_size}.npy!, shape={embeddings.shape}')
            embeddings = None
            embeddings = []
        
    with torch.no_grad():
        model.eval()
        
        val_loss = 0
        val_correct = 0
        val_total = 0

        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

            val_loss += loss.item()
            _, predicted_labels = torch.max(logits, dim=1)
            val_correct += (predicted_labels == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch: {epoch + 1}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    
    if avg_val_loss < min_val_loss:
        # Save the trained model
        model.save_pretrained(MODEL_SAVE_PATH)
        min_val_loss = avg_val_loss
        
    print(f'Epoch {epoch+1} end!!!!!!!')

Epoch 1 start!!!!!!!


 32%|███▏      | 10000/31268 [35:55<256:27:49, 43.41s/it]

Saved ./data/4_embedding/synthesize/secureBERT/embeddings_chunk_0.npy!, shape=(160000, 768)


 64%|██████▍   | 20000/31268 [1:10:15<57:09:42, 18.26s/it]

Saved ./data/4_embedding/synthesize/secureBERT/embeddings_chunk_1.npy!, shape=(160000, 768)


 96%|█████████▌| 30000/31268 [1:44:13<4:58:36, 14.13s/it] 

Saved ./data/4_embedding/synthesize/secureBERT/embeddings_chunk_2.npy!, shape=(160000, 768)


100%|██████████| 31268/31268 [1:48:30<00:00,  4.80it/s]  

Saved ./data/4_embedding/synthesize/secureBERT/embeddings_chunk_3.npy!, shape=(20281, 768)


Epoch: 1, Val Loss: 0.2251, Val Accuracy: 0.9458
Epoch 1 end!!!!!!!


In [14]:
import numpy as np

# Specify the number of chunks you've saved
num_batches = len(train_dataloader)
num_chunks = num_batches // batch_chunk_size + 1 # Replace with the actual number of chunks you've saved

for i in range(num_chunks):
    chunk_path = f'{ENTITY_OUTPUT_PATH}/embeddings_chunk_{i}.npy'
    chunk_embeddings = np.load(chunk_path)
    
    print(f"Chunk {i}: Shape = {chunk_embeddings.shape}")

Chunk 0: Shape = (160000, 768)
Chunk 1: Shape = (160000, 768)
Chunk 2: Shape = (160000, 768)
Chunk 3: Shape = (20281, 768)


In [15]:
160000*6+98479

1058479